In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

## Loading data

In [2]:
list_1 = list()
with open('data/berlin52.txt', 'r') as infile:
    data = infile.readlines()
    for i in data:
        list_1.append(i.split())

cities = int(list_1[0][0])

del list_1[0]

k = 0
for i in range(0, cities):
    k += 1
    for j in range(k, cities):
        list_1[i].append(list_1[j][i])

distance_matrix = np.asarray([list(map(int, x)) for x in list_1])

In [3]:
class Population():
    def __init__(self, bag, distance_matrix):
        self.bag = bag
        self.parents = []
        self.score = 0
        self.best_route = None
        self.distance_matrix = distance_matrix

## Initial population

In [4]:
def init_population(cities, distance_matrix, n_population):
    return Population(np.asarray([random.sample(range(cities), k=cities) for _ in range(n_population)]), distance_matrix)

In [5]:
pop = init_population(cities, distance_matrix, 5)

## Fitting the correct route sequence

In [6]:
def fitness(self, chromosome):
    x = list()
    for i in range(len(chromosome) - 1):
        x.append(self.distance_matrix[chromosome[i], chromosome[i + 1]])
    x.append(self.distance_matrix[chromosome[-1], chromosome[0]])
    return sum(x)

Population.fitness = fitness

## Evaluation of route length

In [7]:
def evaluate(self):
    distances = np.asarray(
        [self.fitness(chromosome) for chromosome in self.bag])
    self.score = min(distances)
    self.best_route = self.bag[distances.tolist().index(self.score)]
    self.parents.append(self.best_route)
    return distances / sum(distances)
    
Population.evaluate = evaluate

In [8]:
pop.evaluate()

array([0.18519945, 0.21012604, 0.19659348, 0.202624  , 0.20545703])

In [9]:
pop.best_route

array([14, 20, 13, 27, 42, 47, 29, 50,  5, 33, 12, 11, 35, 46,  3, 22, 45,
       23, 25, 51, 36, 26, 24, 28, 19, 48, 34, 31,  9, 21, 10,  8,  4, 38,
       16, 40,  1,  7, 32, 15, 49, 37, 41,  6, 44,  2, 30, 18,  0, 39, 17,
       43])

In [10]:
pop.score

26933

## Tournament Selection

In [11]:
def select(self, k=4):
    fit = self.evaluate()
    while len(self.parents) < k:
        idx = random.randint(0, len(fit)-1)
        if fit[idx] > random.random():
            self.parents.append(self.bag[idx])
    self.parents = np.asarray(self.parents)

Population.select = select

## Swap and inversion functions

In [12]:
def swap(chromosome):
    a, b = random.choices(range(len(chromosome)), k=2)
    chromosome[a], chromosome[b] = (chromosome[b], chromosome[a])
    return chromosome

def inversion(chromosome):
    a, b = random.choices(range(len(chromosome)), k=2)
    start, stop = min(a,b), max(a,b)
    for i in range(0, min(len(chromosome[:start]),len(chromosome[stop:]))):
        chromosome[start-1-i], chromosome[stop+i] = chromosome[stop+i], chromosome[start-1-i]
    return chromosome

## PMX crossing

In [13]:
def crossover(self, p_cross=0.1):
    children = []
    count, size = self.parents.shape
    for _ in range(len(self.bag)):
        if random.random() < p_cross:
            children.append(
                list(self.parents[random.choices(range(count), k=1)[0]])
            )
        else:
            parent1, parent2 = self.parents[
                random.choices(range(count), k=2), :
            ]
            idx = random.sample(range(size), k=2)
            start, end = min(idx), max(idx)
            child = [None] * size
            for i in range(start, end + 1, 1):
                child[i] = parent1[i]
            pointer = 0
            for i in range(size):
                if child[i] is None:
                    while parent2[pointer] in child:
                        pointer += 1
                    child[i] = parent2[pointer]
            children.append(child)
    return children

Population.crossover = crossover

## CX crossing

In [14]:
def cycle_crossover(self, p_cx = 0.8):
    children = []
    count, size = self.parents.shape
    for _ in range(len(self.bag)):
        if random.random() < p_cx:
            children.append(list(self.parents[random.choices(range(count), k=1)[0]]))
        else:
            parent1, parent2 = self.parents[random.choices(range(count), k=2), :].tolist()
            cycles = [-1]*len(parent1)
            cycle_no = 1
            cyclestart = (i for i, v in enumerate(cycles) if v < 0)
            for pos in cyclestart:
                while cycles[pos] < 0:
                    cycles[pos] = cycle_no
                    pos = parent1.index(parent2[pos])      
                cycle_no += 1
            child1 = [parent1[i] if n % 2 else parent2[i] for i, n in enumerate(cycles)]
            child2 = [parent2[i] if n % 2 else parent1[i] for i, n in enumerate(cycles)]
            children.append(child1)
    return children


Population.cycle_crossover = cycle_crossover

## Implementation of crossover and mutation

In [33]:
def mutate(self, p_cross=0.1, p_mut=0.1, alg='pmx', mutation='inversion'):
    next_bag = []
    if alg == 'pmx':
        children = self.crossover(p_cross)
    elif alg == 'cx':
        children = self.cycle_crossover(p_cross)
    for child in children:
        if random.random() < p_mut:
            if mutation == 'swap':
                next_bag.append(swap(child))
            else:
                next_bag.append(inversion(child))
        else:
            next_bag.append(child)
    return next_bag
    
Population.mutate = mutate

## Traveling salesman problem - genetic algorithm

In [34]:
def tsp_genetic_algorithm(
    cities,
    distance_matrix,
    n_population=5,
    n_iter=3000,
    selectivity=0.15,
    first_alg = 'pmx',
    second_alg = 'cx',
    p_cross=0.5,
    p_mut=0.1,
    mutation='inversion',
    interval=100,
    change_alg = 1/2,
    use_plot=False):
    
    pop = init_population(cities, distance_matrix, n_population)
    best_route = pop.best_route
    score = float("inf")
    scores = []
    algorithms = [first_alg, second_alg]
    for i in range(n_iter):
        pop.select(n_population * selectivity)
        scores.append(pop.score)
        if (i % interval == 0) and (use_plot==False):
            print(i, "-".join(map(str, pop.best_route)), pop.score)
        if pop.score < score:
            best_route = pop.best_route
            score = pop.score
        if i % 1000 == 0:
            rng = random.randint(0, 1)
        children = pop.mutate(p_cross, p_mut, algorithms[rng], mutation)
        pop = Population(children, pop.distance_matrix)
    if use_plot:
        plt.plot(range(len(scores)), scores, color="blue")
        plt.show()
        return scores
    return "-".join(map(str, best_route)), score

In [35]:
scores = tsp_genetic_algorithm(cities, distance_matrix, n_population=28, n_iter=10000, p_cross=0.8, p_mut=0.1, selectivity=0.1,
                  first_alg='pmx', second_alg='cx', mutation='inversion')

0 0-49-42-36-14-15-48-26-4-25-37-27-17-50-41-22-33-35-30-3-11-38-23-31-8-21-16-6-40-2-45-44-20-29-28-34-7-19-24-10-12-47-1-9-32-18-43-5-51-46-13-39 26516
100 6-16-17-0-43-36-15-48-5-18-32-9-44-2-40-8-25-14-23-39-42-7-34-21-20-29-28-45-30-37-47-12-10-24-13-11-38-31-19-49-27-50-4-26-51-46-3-35-33-22-41-1 18846
200 1-6-16-17-0-43-19-49-27-50-25-14-23-39-34-38-18-32-9-44-40-8-3-24-11-13-46-12-10-35-33-22-45-15-36-20-21-31-2-30-7-42-51-26-37-47-4-5-48-28-29-41 16432
300 6-1-41-29-28-19-49-27-26-51-42-7-33-35-10-12-46-13-11-25-14-23-39-34-38-18-32-9-44-40-8-3-24-50-20-21-31-2-30-22-45-15-36-37-47-4-5-48-43-0-17-16 15795
400 41-1-6-16-17-0-43-48-5-4-47-37-36-15-45-22-30-2-31-21-20-40-50-24-3-8-32-9-44-18-38-34-39-23-14-25-11-13-46-12-10-35-33-7-42-51-26-27-49-19-28-29 15683
500 41-1-29-28-19-49-27-26-51-42-7-33-35-10-12-46-13-11-25-14-23-39-34-38-18-44-32-9-8-3-24-50-40-20-21-31-2-30-22-45-15-36-37-47-4-5-48-43-0-17-16-6 15661
600 16-17-0-48-37-47-4-5-36-15-45-22-30-2-31-21-20-40-50-14-23-39-

5300 50-10-12-27-25-46-13-51-26-28-19-49-15-43-45-4-37-39-14-3-42-32-9-8-7-40-44-18-2-16-41-6-1-29-22-20-21-17-30-0-48-38-35-34-31-33-36-23-5-47-24-11 9590
5400 50-10-12-27-25-46-13-51-26-28-19-49-15-43-45-4-37-39-14-3-42-32-9-8-7-40-44-18-2-16-41-6-1-29-22-20-21-17-30-0-48-38-35-34-31-33-36-23-5-47-24-11 9590
5500 11-24-47-5-23-36-33-31-34-35-38-48-0-30-17-21-20-22-29-1-6-41-16-2-18-44-40-7-8-9-32-42-3-14-39-37-4-45-43-15-49-19-28-26-51-13-46-25-27-12-10-50 9590
5600 11-24-47-5-23-36-33-31-34-35-38-48-0-30-17-21-20-22-29-1-6-41-16-2-18-44-40-7-8-9-32-42-3-14-39-37-4-45-43-15-49-19-28-26-51-13-46-25-27-12-10-50 9590
5700 11-24-47-5-23-36-33-31-34-35-38-48-0-30-17-21-20-22-29-1-6-41-16-2-18-44-40-7-8-9-32-42-3-14-39-37-4-45-43-15-49-19-28-26-51-13-46-25-27-12-10-50 9590
5800 11-24-47-5-23-36-33-31-34-35-38-48-0-30-17-21-20-22-29-1-6-41-16-2-18-44-40-7-8-9-32-42-3-14-39-37-4-45-43-15-49-19-28-26-51-13-46-25-27-12-10-50 9590
5900 26-27-25-46-28-19-49-15-43-45-4-37-5-23-36-33-31-34-39-14-3

## Verification of correctness of route distance calculation

In [ ]:
import sys
def zbudujMacierz(wiersze):
    macierz = [[0 for col in range(int(wiersze[0]))] for row in range(int(wiersze[0]))]
    i = 0
    for w in wiersze[1:]:
        w = w.strip().split(' ')
        j = 0
        for odl in w:
            macierz[i][j] = int(odl)
            macierz[j][i] = int(odl)
            j+=1
        i+=1
    return macierz

def obliczOdleglosc(trasa, macierz):
    odleglosc = 0
    trasa = trasa.strip().split('-')
    try:
        trasa = list(map(int, trasa))
    except ValueError:
        pass
    for i in range(len(trasa)-1):
        odleglosc+=macierz[trasa[i]][trasa[i+1]]
    odleglosc+=macierz[trasa[len(trasa)-1]][trasa[0]]
    return odleglosc, trasa

In [ ]:
obliczOdleglosc(scores[0], distance_matrix)

#scores[0]
#x = '38-33-34-35-48-21-30-17-2-16-20-41-6-1-29-49-19-22-0-31-44-18-40-7-8-9-42-32-50-11-10-51-13-12-26-27-25-46-28-15-43-45-24-3-5-14-4-23-47-37-36-39'
#obliczOdleglosc(x, distance_matrix)